Статьи были взяты с сайта http://special.kremlin.ru/events/president/news. Ключевые слова были указаны под заголовком каждой статьи.

In [1]:
from pymorphy2 import MorphAnalyzer
from pymorphy2.tokenizers import simple_word_tokenize

In [2]:
m = MorphAnalyzer()
def normalize_text(text):
    lemmas = []
    for t in simple_word_tokenize(text):
        lemmas.append(m.parse(t)[0].normal_form)
    return ' '.join(lemmas)

In [3]:
import os

In [4]:
given_keys = []
my_keys = []
directory = 'C://Users//ASUS//Desktop//автобрея//hw1//keywords'

for root, dirs, files in os.walk(directory):
    for file in files:
        if file.endswith('.txt'):
            
            with open(file, 'r', encoding = 'utf-8') as r:
                lines = r.readlines()
                
                given_norm = []
                my_norm = []
                
                for i in lines[0].split(','):
                    given_norm.append(normalize_text(i))
                for i in lines[1].split(','):
                    my_norm.append(normalize_text(i))
                    
                given_keys.append(given_norm)
                my_keys.append(my_norm)

In [5]:
print('Пересечения ключевых слов' + '\n')
for i in range(5):
    given_set = set(given_keys[i])
    my_set = set(my_keys[i])
    print('text ' + str(i+1) + ': ' + str(len(given_set.intersection(my_set))))

Пересечения ключевых слов

text 1: 1
text 2: 2
text 3: 1
text 4: 2
text 5: 3


In [18]:
#Подгружаем стоп-слова
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
stop = stopwords.words('russian')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## 1. TextRank 

Сначала попробуем с gensim

In [96]:
def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d', '', text)
    text = re.sub(r'[A-Za-z]', '', text)
    text = [m.normal_forms(i)[0] for i in text.split() if i not in
            stop]
    return text

In [6]:
from gensim.summarization import keywords as kw

C:\Users\ASUS\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [104]:
gensim_kws = []

for i in range(1, 6):
    filename = str(i) + '.txt'
    
    with open(filename, 'r', encoding = 'utf-8') as r:
        text = r.read()
        text = ' '.join(clean_text(text))
        gensim_kws.append(kw(normalize_text(text), pos_filter=[], scores=True)[:7])

In [105]:
gensim_kws

[[('деловои форум брикс', 0.17694887281538338),
  ('начать год', 0.15433564850462436),
  ('россия многие', 0.1367563685740837),
  ('страна', 0.13316559100865638),
  ('государство', 0.125403674155992),
  ('связь наш экономика это', 0.11943007939991718),
  ('уважаемыи', 0.11593210593825833)],
 [('сделать', 0.19663128583421613),
  ('сегодня', 0.17716243803523324),
  ('ближнии восток', 0.17565883773209792),
  ('встреча глава', 0.16728398300112274),
  ('святеишество наш', 0.1410912187594509),
  ('самыи', 0.13713446295328677),
  ('хотеть', 0.13690381649612418)],
 [('это', 0.3527745441509442),
  ('год москва', 0.1633502909309378),
  ('производство', 0.14024169987641186),
  ('два', 0.12432845566346629),
  ('ивановскии область', 0.1193391326428287),
  ('создать', 0.10492563685012488),
  ('программа воссоздание малое город россия впутин', 0.10449826813057499)],
 [('портовыи', 0.19724195976497286),
  ('комплекс', 0.19073828088640882),
  ('портовоиндустриальныи парк отэко', 0.17517567827536493),
 

Теперь с библиотекой summa

In [16]:
from summa import keywords

In [22]:
summa_kws = []

for i in range(1, 6):
    filename = str(i) + '.txt'
    
    with open(filename, 'r', encoding = 'utf-8') as r:
        text = r.read()
        summa_kws.append(keywords.keywords(normalize_text(text), additional_stopwords=stop, scores=True)[:7])

In [23]:
summa_kws

[[('наш экономика', 0.16281698481187268),
  ('деловой форум брикс', 0.16215588415490703),
  ('год темп', 0.15756788917288161),
  ('экономический', 0.14926596329170674),
  ('россия', 0.14886844530331236),
  ('страна', 0.13127923376158268),
  ('развитие', 0.1298754807134002)],
 [('наш', 0.2504094292681494),
  ('самый', 0.16854907902946312),
  ('ваш', 0.16739014642745928),
  ('встреча', 0.16670463502333177),
  ('это', 0.16429600329964805),
  ('который', 0.14523451614266977),
  ('должный сделать', 0.13211780578182486)],
 [('это', 0.3938784369449436),
  ('год', 0.2169150907224778),
  ('предприятие', 0.13027575427522525),
  ('весь', 0.12540142525555972),
  ('производство', 0.12134351266343057),
  ('который', 0.12027365119516274),
  ('программа воссоздание малое город россия', 0.10775277011909908)],
 [('отэко', 0.23192687581787788),
  ('инфраструктура', 0.19381677198081457),
  ('портовый', 0.1913151303382091),
  ('средство', 0.18951566949670418),
  ('запланировать', 0.17861866883270625),
  ('

## 2. RAKE

In [25]:
import RAKE

In [26]:
# Инициализируем анализатор списком стоп-слов
rake = RAKE.Rake(stop)

In [32]:
rake_kws = []

for i in range(1, 6):
    filename = str(i) + '.txt'
    
    with open(filename, 'r', encoding = 'utf-8') as r:
        text = r.read()
        rake_kws.append(rake.run(normalize_text(text), maxWords=3, minFrequency=2))

In [33]:
rake_kws

[[('брикс', 1.875),
  ('хотеть', 1.6666666666666667),
  ('россия', 1.6666666666666667),
  ('партнёр', 1.5),
  ('весь', 1.3333333333333333),
  ('прежде', 1.3333333333333333),
  ('инвестиция', 1.0)],
 [('ближний восток', 4.333333333333333),
  ('наш встреча', 3.761904761904762),
  ('это регион', 3.35),
  ('весь', 1.75),
  ('спросить', 1.6666666666666667),
  ('поэтому', 1.6666666666666667),
  ('целое', 1.6666666666666667),
  ('знать', 1.6666666666666667),
  ('это', 1.6),
  ('встреча', 1.4285714285714286),
  ('мочь', 1.4),
  ('хотеть', 1.3333333333333333),
  ('сегодня', 1.3333333333333333),
  ('происходить', 1.0),
  ('известно', 1.0),
  ('борьба', 1.0),
  ('несмотря', 1.0),
  ('беспокоить', 1.0)],
 [('прекрасный русский город', 8.3),
  ('это год', 4.4),
  ('речь идти', 4.0),
  ('владимир владимир', 4.0),
  ('во-первых', 4.0),
  ('очень важный', 4.0),
  ('ивановский область', 3.95),
  ('город иваново', 3.8),
  ('мочь сказать', 3.666666666666667),
  ('это', 2.066666666666667),
  ('создать', 2

## 3. TF-IDF 

In [36]:
import re

In [39]:
def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d', '', text)
    text = re.sub(r'[A-Za-z]', '', text)
    text = [m.normal_forms(i)[0] for i in text.split() if i not in
            stop]
    return text

In [40]:
corpus = []
for i in range(1, 6):
    filename = str(i) + '.txt'
    
    with open(filename, 'r', encoding = 'utf-8') as r:
        text = r.read()
        corpus.append(' '.join(clean_text(text)))

In [44]:
import numpy as np
from math import log

In [45]:
def index(corpus):
    X = vectorizer.fit_transform(corpus)
    #матрица с количеством вхождений слов
    matrix = X.toarray()
    #обратная матрица
    rev_matrix = np.transpose(matrix)
    words = vectorizer.get_feature_names()
    return words, rev_matrix

In [68]:
words, rev_matrix = index(corpus)

In [83]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()

In [94]:
n = 7
tfidf_kws = []

for doc in corpus:
    matrix = tfidf.fit_transform(doc.split())
    feature_array = np.array(tfidf.get_feature_names())
    
    tfidf_sorting = np.argsort(matrix.toarray()).flatten()[::-1]
    top_n = feature_array[tfidf_sorting][:n]
    tfidf_kws.append(top_n)

In [106]:
tfidf_kws

[array(['спасибо', 'являться', 'мирный', 'макроэкономический',
        'максимально', 'малое', 'масштабный'], dtype='<U21'),
 array(['добрый', 'являться', 'иерарх', 'мой', 'много', 'многий', 'многие'],
       dtype='<U16'),
 array(['выглядеть', 'январь', 'краткий', 'комплексный', 'комфортный',
        'конечно', 'конечный'], dtype='<U22'),
 array(['россия', 'являться', 'инвестор', 'инфраструктура', 'кластер',
        'компания', 'комплекс'], dtype='<U21'),
 array(['жээнбеков', 'этап', 'донгузый', 'около', 'область', 'наступление',
        'на'], dtype='<U15')]

## Точность, полнота, F-мера

In [107]:
from sklearn.metrics import precision_recall_fscore_support

TextRank: gensim

In [115]:
#Сохраняю только ключевые слова без числовых значений 
gensim = []

for i in gensim_kws:
    arr = []
    for j in i:
        arr.append(j[0])
    gensim.append(arr)

In [128]:
#Точность, полнота, f-мера и support относительно моих ключевых слов
n = 0
for i, j in zip(my_keys, gensim):
    n += 1
    print('text ' + str(n) + ': ' + str(precision_recall_fscore_support(i, j[:len(i)], average='micro')))

text 1: (0.0, 0.0, 0.0, None)
text 2: (0.0, 0.0, 0.0, None)
text 3: (0.0, 0.0, 0.0, None)
text 4: (0.0, 0.0, 0.0, None)
text 5: (0.0, 0.0, 0.0, None)


In [130]:
#Точность, полнота, f-мера и support относительно данных ключевых слов
n = 0
for i, j in zip(given_keys, gensim):
    n += 1
    print('text ' + str(n) + ': ' + str(precision_recall_fscore_support(i, j[:len(i)], average='micro')))

text 1: (0.0, 0.0, 0.0, None)
text 2: (0.0, 0.0, 0.0, None)
text 3: (0.0, 0.0, 0.0, None)
text 4: (0.0, 0.0, 0.0, None)
text 5: (0.0, 0.0, 0.0, None)


TextRank: summa

In [119]:
#Сохраняю только ключевые слова без числовых значений 
summa = []

for i in summa_kws:
    arr = []
    for j in i:
        arr.append(j[0])
    summa.append(arr)

In [131]:
#Точность, полнота, f-мера и support относительно моих ключевых слов
n = 0
for i, j in zip(my_keys, summa):
    n += 1
    print('text ' + str(n) + ': ' + str(precision_recall_fscore_support(i, j[:len(i)], average='micro')))

text 1: (0.0, 0.0, 0.0, None)
text 2: (0.0, 0.0, 0.0, None)
text 3: (0.0, 0.0, 0.0, None)
text 4: (0.0, 0.0, 0.0, None)
text 5: (0.16666666666666666, 0.16666666666666666, 0.16666666666666666, None)


In [138]:
#Точность, полнота, f-мера и support относительно данных ключевых слов
n = 0
for i, j in zip(given_keys, summa):
    n += 1
    print('text ' + str(n) + ': ' + str(precision_recall_fscore_support(i, j[:len(i)], average='micro')))

text 1: (0.0, 0.0, 0.0, None)
text 2: (0.0, 0.0, 0.0, None)
text 3: (0.0, 0.0, 0.0, None)
text 4: (0.0, 0.0, 0.0, None)
text 5: (0.0, 0.0, 0.0, None)


RAKE

In [134]:
#Сохраняю только ключевые слова без числовых значений 
rake = []

for i in rake_kws:
    arr = []
    for j in i:
        arr.append(j[0])
    rake.append(arr)

In [140]:
#Точность, полнота, f-мера и support относительно моих ключевых слов
n = 0
for i, j in zip(my_keys, rake):
    n += 1
    if len(i) >= len(j):
        ii = i[:len(j)]
        jj = j
    else:
        jj = j[:len(i)]
        ii = i
    print('text ' + str(n) + ': ' + str(precision_recall_fscore_support(ii, jj, average='micro')))

text 1: (0.0, 0.0, 0.0, None)
text 2: (0.0, 0.0, 0.0, None)
text 3: (0.0, 0.0, 0.0, None)
text 4: (0.0, 0.0, 0.0, None)
text 5: (0.0, 0.0, 0.0, None)


C:\Users\ASUS\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\ASUS\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


In [141]:
#Точность, полнота, f-мера и support относительно данных ключевых слов
n = 0
for i, j in zip(given_keys, rake):
    n += 1
    if len(i) >= len(j):
        ii = i[:len(j)]
        jj = j
    else:
        jj = j[:len(i)]
        ii = i
    print('text ' + str(n) + ': ' + str(precision_recall_fscore_support(ii, jj, average='micro')))

text 1: (0.25, 0.25, 0.25, None)
text 2: (0.0, 0.0, 0.0, None)
text 3: (0.0, 0.0, 0.0, None)
text 4: (0.0, 0.0, 0.0, None)
text 5: (0.0, 0.0, 0.0, None)


TF-IDF

In [144]:
#Точность, полнота, f-мера и support относительно моих ключевых слов
n = 0
for i, j in zip(my_keys, tfidf_kws):
    n += 1
    print('text ' + str(n) + ': ' + str(precision_recall_fscore_support(i, j[:len(i)], average='micro')))

text 1: (0.0, 0.0, 0.0, None)
text 2: (0.0, 0.0, 0.0, None)
text 3: (0.0, 0.0, 0.0, None)
text 4: (0.0, 0.0, 0.0, None)
text 5: (0.0, 0.0, 0.0, None)


In [145]:
#Точность, полнота, f-мера и support относительно данных ключевых слов
n = 0
for i, j in zip(given_keys, tfidf_kws):
    n += 1
    print('text ' + str(n) + ': ' + str(precision_recall_fscore_support(i, j[:len(i)], average='micro')))

text 1: (0.0, 0.0, 0.0, None)
text 2: (0.0, 0.0, 0.0, None)
text 3: (0.0, 0.0, 0.0, None)
text 4: (0.0, 0.0, 0.0, None)
text 5: (0.0, 0.0, 0.0, None)


## Выводы

Все использованные методы автоматического выделения ключевых слов справились плохо. Ненулевые значения точности, полноты и F-меры (и то лишь для одного текста) присутствуют только в RAKE и TextRank. Какие могут быть тому причины?

1. Tf-idf не выделяет n-граммы, это могло стать причиной того, что в результатах этого метода одни нули.
2. Возможно, используемые тексты слишком длинные для данных методов.
3. Ключевые слова, предложенные на сайте, откуда были взяты тексты, больше похожи на теги. Однако, это не объясняет нулевой корреляции результатов с ключевыми словами, выделенными мной.